# Асинхронность

Понятия асинхронности и многопоточности часто путаются. При этом, если многопоточность - это средство для параллельного выполнения кода, то асинхронность - это средство для организации неблокирующего взаимодействия между разными частями программы. Причем, асинхронность может быть реализована как с помощью многопоточности, так и без нее.

Асинхронность еще можно назвать парадигмой программирования, в которой разработчик должен обеспечивать неблокирующее взаимодействие между разными частями программы. При этом количество потоков, их утилизация и т.д. - это уже вопросы для соответствующих фреймворков.

Реализация инструментов и фреймворков такого рода - это сложная задача, поэтому всегда найдутся какие-то нюансы, которые необходимо учитывать и понимать как они работают.

Так какие-же средства предоставляет нам dotnet для реализации асинхронности?

In [ ]:
using System.Threading;
using System.Threading.Tasks;
using System.Diagnostics;

private static int counter;
private const int OperationDuration = 2000;

private static int MaxThreads
{
    get
    {
        ThreadPool.GetMaxThreads(out var workerThreads, out _);
        return workerThreads;
    }
}
private static int AvailableThreads
{
    get
    {
        ThreadPool.GetAvailableThreads(out var workerThreads, out _);
        return workerThreads;
    }
}

static void Main()
{
    ThreadPool.SetMaxThreads(10, 10);
    Console.WriteLine($"Starting with thread capacity {MaxThreads}");
    var timer = new Timer(TimerTick);

    Console.WriteLine("Performing operation...");
    var sw = new Stopwatch();
    sw.Start();
    DoOperationAsync().Wait();
    sw.Stop();
    Console.WriteLine($"Single operation completes in {sw.ElapsedMilliseconds} ms.");

    timer.Change(0, 1000);
    Thread.Sleep(10000);
    Console.WriteLine($"Operations completed: {counter}");
}

private static void TimerTick(object state)
{
    for (int i = 0; i < 100; i++)
    {
        DoOperationAsync();
    }
    Console.WriteLine($"Added another 100 operations. Threads available {AvailableThreads}");
}

private static async Task DoOperationAsync()
{
    await Task.Yield();
    Thread.Sleep(OperationDuration);
    //await Task.Delay(OperationDuration);
    Interlocked.Increment(ref counter);
}

Main()

# Async I/O

In [ ]:
using System;
using System.IO;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

private const int BUFFER_SIZE = 4096;

private static void Main(string[] args)
{
    Task t = ProcessAsynchronousIO();
    t.GetAwaiter().GetResult();
}

private static async Task ProcessAsynchronousIO()
{
    using (var stream = new FileStream("test1.txt", FileMode.Create,
        FileAccess.ReadWrite, FileShare.None, BUFFER_SIZE))
    {
        Console.WriteLine("1. Uses I/O Threads: {0}", stream.IsAsync);

        byte[] buffer = Encoding.UTF8.GetBytes(CreateFileContent());
        Task t = stream.WriteAsync(buffer, 0, buffer.Length);
        await t;
    }

    using (var stream = new FileStream("test2.txt", FileMode.Create, FileAccess.ReadWrite,
        FileShare.None, BUFFER_SIZE, FileOptions.Asynchronous))
    {
        Console.WriteLine("2. Uses I/O Threads: {0}", stream.IsAsync);

        byte[] buffer = Encoding.UTF8.GetBytes(CreateFileContent());
        Task t = stream.WriteAsync(buffer, 0, buffer.Length);
        await t;
    }

    using (FileStream stream = File.Create("test3.txt", BUFFER_SIZE, FileOptions.Asynchronous))
    {
        using (var sw = new StreamWriter(stream))
        {
            Console.WriteLine("3. Uses I/O Threads: {0}", stream.IsAsync);
            await sw.WriteAsync(CreateFileContent());
        }
    }

    using (var sw = new StreamWriter("test4.txt", true))
    {
        Console.WriteLine("4. Uses I/O Threads: {0}", ((FileStream)sw.BaseStream).IsAsync);
        await sw.WriteAsync(CreateFileContent());
    }

    Console.WriteLine("Starting parsing files in parallel");

    var readTasks = new Task<long>[4];
    for (int i = 0; i < 4; i++)
    {
        readTasks[i] = SumFileContent(string.Format("test{0}.txt", i + 1));
    }

    long[] sums = await Task.WhenAll(readTasks);

    Console.WriteLine("Sum in all files: {0}", sums.Sum());

    Console.WriteLine("Deleting files...");

    var deleteTasks = new Task[4];
    for (int i = 0; i < 4; i++)
    {
        string fileName = string.Format("test{0}.txt", i + 1);
        deleteTasks[i] = SimulateAsynchronousDelete(fileName);
    }

    await Task.WhenAll(deleteTasks);

    Console.WriteLine("Deleting complete.");
}

private static string CreateFileContent()
{
    var sb = new StringBuilder();
    for (int i = 0; i < 100000; i++)
    {
        sb.AppendFormat("{0}", new Random(i).Next(0, 99999));
        sb.AppendLine();
    }
    return sb.ToString();
}

private static async Task<long> SumFileContent(string fileName)
{
    using (var stream = new FileStream(fileName, FileMode.Open,
        FileAccess.Read, FileShare.None, BUFFER_SIZE, FileOptions.Asynchronous))
    using (var sr = new StreamReader(stream))
    {
        long sum = 0;
        while (sr.Peek() > -1)
        {
            string line = await sr.ReadLineAsync();
            sum += long.Parse(line);
        }

        return sum;
    }
}

private static Task SimulateAsynchronousDelete(string fileName)
{
    return Task.Run(() => File.Delete(fileName));
}